In [1]:
%reload_ext autoreload
%autoreload 2
%cd llama3/llama

/home/tbai4/llama3/llama


/home/tbai4/llama3/.venv/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [ ]:
import os
from llama import Workflow, Llama
from llama.util import load_model_and_tokenizer

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=12288,
    max_batch_size=2,
    model_parallel_size=1,
    max_nodes=20,
    use_lora=True,
    lora_rank=8,
    lora_alpha=16,
    lora_dropout=0.1
)

workflow.model.get_trainable_param_percentage()

/home/tbai4/llama3/llama/util.py:55: UserWarning: 12288 does not lie within [1, 8192]
  warnings.warn(f"{max_seq_len} does not lie within [1, 8192]")


> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1


In [11]:
import torch
from llama.workflows.finetune import TotTrainer
from llama.workflows.tot import cot_prompt, finish_prompt, format_vote_system_prompt, format_problem

sample = torch.load('tot_data/problem_0.pt', weights_only=True)
trainer = TotTrainer(workflow, branching_factor=8, voters=4)

cot, vote, finish = workflow.insert([
    {'messages': [
        {'role': 'system', 'content': cot_prompt},
        {'role': 'user', 'content': format_problem(sample['problem'])}
    ], 'parent_ids': []},
    {'messages': [
        {'role': 'system', 'content': format_vote_system_prompt(8)},
        {'role': 'user', 'content': format_problem(sample['problem'])}
    ], 'parent_ids': []},
    {'messages': [
        {'role': 'system', 'content': finish_prompt},
        {'role': 'user', 'content': format_problem(sample['problem'])}
    ], 'parent_ids': []},
], training=True)

proposal_tasks = [
    {'header': ('assistant', None),
     'prefill': f'Solution #{i+1}:\n\n',
     'parent_ids': [cot['id']]}
    for i in range(8)
]
target_proposal_ids = [p + [workflow.tokenizer.eot_id] for p in sample['result']['proposal_tokens']]
proposal_nodes, proposal_logprobs = workflow.train_step(proposal_tasks, target_proposal_ids)

Training 6.8M / 8.0B parameters


In [15]:
workflow.reset()
workflow.model.set_adapter_state(enabled=False)

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer the user\'s question please.'}],
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
])

[output], _ = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [system['id'], user_1['id']],
        }
    ]
)

workflow.tokenizer.decode(output)

'The capital of France is Paris.'

In [14]:
workflow.reset()
workflow.model.set_adapter_state(enabled=True)

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer the user\'s question please.'}],
        'parent_ids': [],
    },
])

[user_1] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
])

[output], _ = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [system['id'], user_1['id']],
        }
    ]
)

workflow.tokenizer.decode(output)

'The capital of France is Paris.'

## sanity check forward-backward

In [5]:
import torch
from llama.workflows.finetune import TotTrainer
from llama.workflows.tot import cot_prompt, finish_prompt, format_vote_system_prompt, format_problem

problem = torch.load('tot_data/problem_0.pt', weights_only=True)
trainer = TotTrainer(workflow, branching_factor=8, voters=4)

workflow.model.train()
workflow.model.set_adapter_state(enabled=True)
workflow.model.zero_grad()

total_loss, metrics = trainer.step(problem)

metrics

Training 6.8M / 8.0B parameters


In [ ]:
from llama.workflows.finetune import finetune

finetune(
    data_path='/home/tbai4/llama3/llama/tot_data_2',
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    output_dir='/scratch4/jeisner1/tjbai/checkpoints/tot_2',
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    max_seq_len=6144,
    checkpoint_freq=100,
    validation_freq=100,
)

## load and run checkpoints

In [ ]:
import torch

checkpoint = torch.load('/scratch4/jeisner1/tjbai/checkpoints/lora_epoch-0_step-399.pt', weights_only=True)
workflow.model.load_state_dict(checkpoint['lora'])

workflow.reset()
workflow.model.eval()
workflow.model.set_adapter_state(True)

[system] = workflow.insert([
    {
        'messages': [{'role': 'system', 'content': 'Answer ALL of the user\'s question(s).'}],
        'parent_ids': [],
    },
])

[user_1, user_2] = workflow.insert([
    {
        'messages': [{'role': 'user', 'content': 'What is the capital of France?'}],
        'parent_ids': [system['id']],
    },
    {
        'messages': [{'role': 'user', 'content': 'What is the largest planet in the solar system?'}],
        'parent_ids': [system['id']],
    },
])

[output], _ = workflow.step(
    [
        {
            'header': ('assistant', None),
            'prefill': '',
            'parent_ids': [system['id'], user_1['id'], user_2['id']],
        }
    ],
)

workflow.tokenizer.decode(output)

## evaluate trick prompt results across checkpoints

In [ ]:
import torch
import json
import random
import random
from llama import Llama
from llama.workflows.tot import load_math_problems, benchmark_tricky_tot
from tqdm import tqdm

problems = load_math_problems('../data/MATH', split='val')
problems = random.sample(problems, 200)

for id in [99, 199, 299, 399]: 
    checkpoint = torch.load(f'/scratch4/jeisner1/tjbai/checkpoints/lora_epoch-0_step-{id}.pt', weights_only=True)
    workflow.model.load_state_dict(checkpoint['lora'])
    llama = Llama(workflow.model, workflow.tokenizer)
    print(f'Loaded checkpoint-{id}')
    print(f'Memory allocated: {torch.cuda.memory_allocated()}')

    comps = []
    for problem in tqdm(problems):
        comps.append(benchmark_tricky_tot(
            llama=llama,
            workflow=workflow,
            problem=problem['problem'],
            branching_factor=8,
            voters=4
        ))
        
    with open(f'checkpoint-{id}_trick_results.json', 'w') as f:
        json.dump(comps, f)

## generate and evaluate final solutions

In [ ]:
import torch
import json
import random
from llama import Llama
from llama.workflows.tot import load_math_problems, benchmark_solution_quality
from tqdm import tqdm

random.seed(42)
problems = load_math_problems('../data/MATH', split='val')
problems = random.sample(problems, 200)

for id in [99, 199, 299, 399]: 
    checkpoint = torch.load(f'/scratch4/jeisner1/tjbai/checkpoints/lora_epoch-0_step-{id}.pt', weights_only=True)
    workflow.model.load_state_dict(checkpoint['lora'])
    llama = Llama(workflow.model, workflow.tokenizer)
    print(f'Loaded checkpoint-{id}')
    print(f'Memory allocated: {torch.cuda.memory_allocated()}')

    comps = []
    for problem in tqdm(problems):
        comps.append(benchmark_solution_quality(
            llama=llama,
            workflow=workflow,
            problem=problem['problem'],
            branching_factor=8,
            voters=4,
            compact=False,
        ))
        
    with open(f'checkpoint-{id}_solution_quality.json', 'w') as f:
        json.dump(comps, f)
        
# TODO -- get the untrained version's results too

In [7]:
import os
from llama import Llama

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

llama = Llama.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=4,
    model_parallel_size=1,
)

import re
import json
import random
from tqdm import tqdm
from collections import Counter
from llama.workflows.tot import load_math_problems, benchmark_solution_quality, parse_choice

evaluator_prompt = '''
You are evaluating final answers to AMC/AIME competition problems. You will receive:

1. A problem statement
2. The ground truth solution 
3. A shared solution proposal that both contestants used
4. Two final answers based on this proposal

Your task is to evaluate how effectively each contestant converted the shared proposal into a valid solution.
Note that valid solutions may differ from the ground truth approach while remaining correct.

Evaluate both answers focusing on:
1. Answer Format Quality
- Clarity and conciseness of final statement
- Proper mathematical notation
- Inclusion of key numerical result

2. Mathematical Validity
- Correctness of final numerical answer
- Completeness (all parts answered)
- Any invalid mathematical claims

3. Justification Level
- Appropriate amount of supporting context
- Balance between brevity and explanation
- Clear connection to previous reasoning

Walk through each of these criterion and compare the 2 solutions. 

You must format your response as:

VERDICT: [1 or 2]
VERDICT_NOTE: (one sentence explanation)
'''

random.seed(42)
problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='val')
problems = random.sample(problems, 200)

for id in [0, 99, 199, 299, 399]:
    with open(f'/home/tbai4/llama3/dumps/checkpoint-{id}_solution_quality.json', 'r') as f:
        data = json.load(f)

    baseline_win = 0
    cached_win = 0

    for d, problem_obj in tqdm(zip(data, problems), total=200):
        problem = d['problem']
        solution = problem_obj['solution']
        baseline_final = d['baseline_final']
        cached_final = d['cached_final']

        votes = [
            choice for resp in d['voters'] if
            (choice := parse_choice(resp)) is not None
        ]
        best = Counter(votes).most_common(1)[0][0] - 1

        baseline_first = random.choice([True, False])
        ans1 = baseline_final if baseline_first else cached_final 
        ans2 = cached_final if baseline_first else baseline_final

        dialog = [
            {'role': 'system', 'content': evaluator_prompt},
            {'role': 'user', 'content': f'''
PROBLEM STATEMENT:
{problem}

GROUND TRUTH SOLUTION:
{solution}

SOLUTION PROPOSAL:
{d['proposals'][best]}

FINAL ANSWER #1:
{ans1}

FINAL ANSWER #2:
{ans2}
'''
            }
        ]

        outputs = llama.chat_completion(
            [dialog for _ in range(3)],
            max_gen_len=256,
            temperature=0.7,
            top_p=0.9,
            seed=42,
        )
        
        baseline_vote = 0
        cached_vote = 0
        for evaluation in outputs:
            match = re.search(r'VERDICT:\s*(\d)', evaluation['generation']['content'])
            if match:
                num = int(match.group(1))
                if (num == 1 and baseline_first) or (num == 2 and not baseline_first):
                    baseline_vote += 1
                else:
                    cached_vote += 1
        
        baseline_win += baseline_vote > cached_vote
        cached_win += cached_vote > baseline_vote
                
    print(baseline_win, cached_win)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 13.94 seconds


100%|██████████| 200/200 [13:22<00:00,  4.01s/it]


137 63


100%|██████████| 200/200 [13:23<00:00,  4.02s/it]


98 102


...

99 101


100%|██████████| 200/200 [13:52<00:00,  4.16s/it]


92 108


100%|██████████| 200/200 [13:53<00:00,  4.17s/it]

107 93


In [2]:
import os
import torch
from llama import Workflow
from llama.workflows.tot import tot_cached, load_math_problems

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

workflow = Workflow.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=8192,
    max_batch_size=4,
    model_parallel_size=1,
    max_nodes=20,
    use_lora=True,
    lora_rank=8,
    lora_alpha=16,
    lora_dropout=0.1
)

checkpoint = torch.load(f'/scratch4/jeisner1/tjbai/checkpoints/lora_epoch-0_step-99.pt', weights_only=True)
workflow.model.load_state_dict(checkpoint['lora'])

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Converting to LoRA
Loaded in 15.68 seconds


<All keys matched successfully>

In [32]:
problems = load_math_problems(
    '../data/MATH',
    split='train',
    problem_types=['counting_and_probability']
)[:200]

from llama import Llama
from llama.workflows.tot import benchmark_solution_quality

llama = Llama(workflow.model, workflow.tokenizer)

outputs = benchmark_solution_quality(
    llama=llama,
    workflow=workflow,
    problem=problems[0]['problem'],
    branching_factor=8,
    voters=4,
    compact=False,
)

In [2]:
import os
import json
from llama import Llama
from llama.workflows.tot import load_math_problems
from tqdm import tqdm

os.environ["RANK"] = "0"
os.environ["WORLD_SIZE"] = "1"
os.environ["MASTER_ADDR"] = "localhost"
os.environ["MASTER_PORT"] = "29500"

llama = Llama.build(
    ckpt_dir='/scratch4/jeisner1/tjbai/llama_8b',
    tokenizer_path='/scratch4/jeisner1/tjbai/llama_8b/tokenizer.model',
    max_seq_len=4096,
    max_batch_size=3,
    model_parallel_size=1,
)

evaluator_prompt = '''
You are a strict evaluator for mathematics problems. You will assess:
1. Problem statement 
2. Official solution and final answer
3. Student's attempted solution and final answer

Evaluation criteria:
- Final answers must be mathematically equivalent to the official solution
- All valid equivalent expressions are correct (e.g., 1/2 vs 0.5 vs 2^-1)

Output: Respond with ONLY "correct" or "incorrect" based on the final answer.
'''

problems = load_math_problems('/home/tbai4/llama3/data/MATH', split='val')

all = []

for path in [
    '/home/tbai4/llama3/dumps/tot_2/hotswap-epoch-0_step-195.json',
    '/home/tbai4/llama3/dumps/tot_2/hotswap-epoch-0_step-395.json',
    '/home/tbai4/llama3/dumps/tot_2/hotswap-epoch-0_step-595.json',
    '/home/tbai4/llama3/dumps/tot_2/hotswap-epoch-0_step-795.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-0_step-195.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-0_step-395.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-0_step-595.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-1_step-95.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-1_step-495.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/tot_2/lora_epoch-1_step-895.pt_e2e.json',
    # '/home/tbai4/llama3/dumps/baseline_e2e.json',
]:
    with open(path) as f:
        data = json.load(f)
    
    correct = []
    for d, problem_obj in tqdm(zip(data, problems), total=280):
        problem = problem_obj['problem']
        solution = problem_obj['solution']
        attempt = llama.tokenizer.decode(d['final_tokens'])
        
        dialog = [
            {'role': 'system', 'content': evaluator_prompt},
            {'role': 'user', 'content': f'''
PROBLEM STATEMENT:
{problem}

GROUND TRUTH SOLUTION:
{solution}

ATTEMPTED SOLUTION:
{attempt}
'''
            }
        ]
            
        outputs = llama.chat_completion(
            [dialog for _ in range(3)],
            max_gen_len=256,
            temperature=0.25,
            top_p=0.9,
            seed=42,
        )
        
        inc = 0
        cor = 0
        for o in outputs:
            if 'incorrect' in o['generation']['content'].lower():
                inc += 1
            elif 'correct' in o['generation']['content'].lower():
                cor += 1
            else:
                print(o['generation']['content'])
                raise Exception()

        correct.append(cor > inc)
        
    print(sum(correct))
    all.append(correct)

> initializing model parallel with size 1
> initializing ddp with size 1
> initializing pipeline with size 1
Loaded in 14.83 seconds


100%|██████████| 280/280 [00:59<00:00,  4.71it/s]


86


100%|██████████| 280/280 [01:00<00:00,  4.60it/s]


82


100%|██████████| 280/280 [01:00<00:00,  4.63it/s]


95


100%|██████████| 280/280 [00:53<00:00,  5.22it/s]

93


In [4]:
import json

with open('tmp') as f:
    all = json.load(f)
    
import numpy as np
from statsmodels.stats.contingency_tables import mcnemar

def run_mcnemars_test(clf1_results, clf2_results):
    clf1_results = np.array(clf1_results)
    clf2_results = np.array(clf2_results)
    
    both_correct = np.sum((clf1_results == True) & (clf2_results == True))
    both_incorrect = np.sum((clf1_results == False) & (clf2_results == False))
    clf1_only = np.sum((clf1_results == True) & (clf2_results == False))
    clf2_only = np.sum((clf1_results == False) & (clf2_results == True))
    
    table = [[both_correct, clf1_only],
             [clf2_only, both_incorrect]]
    
    print(f"Contingency Table:")
    print(f"Both correct: {both_correct}")
    print(f"Baseline correct only: {clf1_only}")
    print(f"Fine-tuned correct only: {clf2_only}")
    print(f"Both incorrect: {both_incorrect}")
    
    # Run test
    result = mcnemar(table, exact=True)  # exact=True for small samples
    
    return result.pvalue

p_value = run_mcnemars_test(all[1], all[0])
print(f"\np-value: {p_value}")

Contingency Table:
Both correct: 70
Baseline correct only: 44
Fine-tuned correct only: 36
Both incorrect: 130

p-value: 0.43404225534547874
